# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model
import numpy as np
import time

In [ ]:
import random
random.seed(42)

# Load InceptionV3 extracted BiLSTM data

In [ ]:
X = np.load(r'/content/drive/My Drive/Model Files/X_train.npy')
y = np.load(r'/content/drive/My Drive/Model Files/y_train.npy')

num_classes = 94
y_one_hot = np.zeros((y.shape[0], y.shape[1], num_classes), dtype=np.int32)
for i in range(y.shape[0]):
    y_one_hot[i] = to_categorical(y[i], num_classes=num_classes)

In [ ]:
X_test = np.load(r'/content/drive/My Drive/Model Files/X_test.npy')
y_test = np.load(r'/content/drive/My Drive/Model Files/y_test.npy')

num_classes = 94
y_test_ohe = np.zeros((y_test.shape[0], y_test.shape[1], num_classes), dtype=np.int32)
for i in range(y_test.shape[0]):
    y_one_hot[i] = to_categorical(y_test[i], num_classes=num_classes)

# Load DCT extracted BiLSTM data

In [ ]:
Xdct = np.load(r'/content/drive/My Drive/Model Files/X_train_300_cropped_dct.npy')
ydct = np.load(r'/content/drive/My Drive/Model Files/y_train_300_cropped_dct.npy')

num_classes = 94
ydct_ohe = np.zeros((ydct.shape[0], ydct.shape[1], num_classes), dtype=np.int32)
for i in range(ydct.shape[0]):
    ydct_ohe[i] = to_categorical(ydct[i], num_classes=num_classes)

In [ ]:
Xdct_test = np.load(r'/content/drive/My Drive/Model Files/X_test_300_cropped_dct.npy')
ydct_test = np.load(r'/content/drive/My Drive/Model Files/y_test_300_cropped_dct.npy')

num_classes = 94
# One-hot encode the entire sequences of labels
ydct_test_ohe = np.zeros((ydct_test.shape[0], ydct_test.shape[1], num_classes), dtype=np.int32)
for i in range(ydct_test.shape[0]):
    ydct_test_ohe[i] = to_categorical(ydct_test[i], num_classes=num_classes)

# Training BiLSTM

In [ ]:
num_classes = 94
# One-hot encode the entire sequences of labels
y_one_hot = np.zeros((ynot1.shape[0], ynot1.shape[1], num_classes), dtype=np.int32)
for i in range(ynot1.shape[0]):
    y_one_hot[i] = to_categorical(ynot1[i], num_classes=num_classes)

In [ ]:
num_classes = 94
# One-hot encode the entire sequences of labels
y_test_hot = np.zeros((y1.shape[0], y1.shape[1], num_classes), dtype=np.int32)
for i in range(y1.shape[0]):
    y_test_hot[i] = to_categorical(y1[i], num_classes=num_classes)

In [ ]:
# Creating Bidrectional LSTM model

epochs = 50
batch_size = 32

max_sequence_length = 5
feature_vector_size = len(X[0][0]) #2048 for IncV3 and 2025 for DCT
model = Sequential()
model.add(Bidirectional(LSTM(units=2048, return_sequences=True), batch_input_shape=(1, max_sequence_length, feature_vector_size)))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X, y_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.2) #callbacks=[model_checkpoint_callback]

Epoch 1/50
191/191 [==============================] - 17s 66ms/step - loss: 3.2447 - accuracy: 0.2237 - val_loss: 2.2474 - val_accuracy: 0.3407
Epoch 2/50
191/191 [==============================] - 11s 59ms/step - loss: 1.5580 - accuracy: 0.5535 - val_loss: 1.2721 - val_accuracy: 0.6282
Epoch 3/50
191/191 [==============================] - 12s 61ms/step - loss: 0.8040 - accuracy: 0.7755 - val_loss: 0.7168 - val_accuracy: 0.7885
Epoch 4/50
191/191 [==============================] - 12s 61ms/step - loss: 0.4463 - accuracy: 0.8839 - val_loss: 0.4733 - val_accuracy: 0.8676
Epoch 5/50
191/191 [==============================] - 11s 58ms/step - loss: 0.2593 - accuracy: 0.9372 - val_loss: 0.4270 - val_accuracy: 0.8781
Epoch 6/50
191/191 [==============================] - 11s 59ms/step - loss: 0.1528 - accuracy: 0.9672 - val_loss: 0.3052 - val_accuracy: 0.9082
Epoch 7/50
191/191 [==============================] - 12s 61ms/step - loss: 0.0838 - accuracy: 0.9871 - val_loss: 0.2245 - val_accuracy:

In [ ]:
model.save(r'/content/drive/My Drive/Model Files/Separated_Best_No6.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
train_results = model.evaluate(X, y_one_hot)
print("Train Loss:", train_results[0])
print("Train Accuracy:", train_results[1])

In [ ]:
train_results = model.evaluate(X_test, y_test_ohe)
print("Train Loss:", train_results[0])
print("Train Accuracy:", train_results[1])

239/239 [==============================] - 5s 20ms/step - loss: 0.0253 - accuracy: 0.9939
Train Loss: 0.025345655158162117
Train Accuracy: 0.9939457178115845


In [ ]:
inference_times = []

for i in range(933):
  input_data = Xdct[-i] #change to i and -i and take the avg
  input_data = np.expand_dims(input_data, axis=0)

  start_time = time.time()
  predictions = model.predict(input_data)
  inference_time = time.time() - start_time
  inference_times.append(inference_time)

average_inference_time = np.mean(inference_times)
print(average_inference_time, "seconds")

# Trying Signer Independence

In [ ]:
model = load_model(r'/content/drive/My Drive/Model Files/Separated_Best_No6.h5')

In [ ]:
X1tr = X[6089:]
y1tr = y[6089:]
Xnot1tr = X[:6089] #np.concatenate((X[:6089],X[6089:]))
ynot1tr = y[:6089] #np.concatenate((y[:6089], y[6089:]))
X1ts = X_test[1542:]
y1ts = y_test[1542:]
Xnot1ts = X_test[:1542] #np.concatenate((X_test[:1542], X_test[1542:]))
ynot1ts = y_test[:1542] #np.concatenate((y_test[:1542], y_test[1542:]))

Xnot1 = np.concatenate((Xnot1tr, Xnot1ts))
ynot1 = np.concatenate((ynot1tr, ynot1ts))
X1 = np.concatenate((X1tr, X1ts))
y1 = np.concatenate((y1tr, y1ts))

print(Xnot1.shape)
print(ynot1.shape)
print(X1.shape)
print(y1.shape)

(7631, 5, 2048)
(7631, 5)
(1512, 5, 2048)
(1512, 5)


In [ ]:
test_results = model.evaluate(X1, y_test_hot)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])
print("WER:", 1 - test_results[1])

48/48 [==============================] - 1s 20ms/step - loss: 4.3481 - accuracy: 0.3881
Test Loss: 4.348137855529785
Test Accuracy: 0.3880952298641205
WER: 0.6119047701358795
